In [2]:
import re
import sqlite3
import pandas as pd
import numpy as np
import nltk
import tqdm as tqdm
import string
from nltk.corpus import stopwords
stop = stopwords.words("english")
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')


from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from math import floor,ceil

from sklearn.svm import LinearSVC

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
import seaborn as sns
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from keras.datasets import imdb
import os
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')

import re
import preprocessor as p

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [7]:
df = pd.read_excel(r'C:\Users\skand\Downloads\interns-tagging_Skandha.xlsx')

In [8]:
df

,cluster_id,id,phrase,common idea
0,0,0334a0d055104e9a931c079e338be9a1,Would use the product again if needed Joe .,loyal customer
1,0,796d6c25ab8849cbba427f1f3e250d80,Have been using the product for a week now,loyal customer
2,0,661f5299cd8944a8a3841fd4f049dee9,Will continue to use this product when I have ...,loyal customer
3,0,da831e4bc58d4505aec3c583f0248f8b,Have always had good luck with this product .,loyal customer
4,0,0ea997675e7344419d1540d3e0bc26c3,Will continue to use This product as This prod...,loyal customer
...,...,...,...,...
599,10,93f874167d11473f8d36d1cda0a0081c,Spray has no strong odor,good smell
600,10,d50fe37fab064408a891aa9ef45dcd70,Spray is nice to keep out on the porch on a su...,good smell
601,10,3e1e8901d5ab4fc9b602ecfdca1220cb,Spray does not leave any oily stinky stains,good smell
602,10,c84e1b1196a242d18938af6c60403afc,Love that the scent of this spray is not chemi...,good smell


In [9]:
df.drop(df.columns[[1,3]], axis=1, inplace=True)

In [10]:
df

,cluster_id,phrase
0,0,Would use the product again if needed Joe .
1,0,Have been using the product for a week now
2,0,Will continue to use this product when I have ...
3,0,Have always had good luck with this product .
4,0,Will continue to use This product as This prod...
...,...,...
599,10,Spray has no strong odor
600,10,Spray is nice to keep out on the porch on a su...
601,10,Spray does not leave any oily stinky stains
602,10,Love that the scent of this spray is not chemi...


## Data PreProcessing & Cleaning

In [11]:
def cleaning( review, remove_stopwords=True):
   
    

    review_text = re.sub("[^a-zA-Z]"," ", review)
   
    words = review_text.lower().split()
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    b=[]
    stemmer = english_stemmer 
    for word in words:
        b.append(stemmer.stem(word))

    
    return(b)

In [14]:
sent = df['phrase']

In [15]:
df

,cluster_id,phrase
0,0,Would use the product again if needed Joe .
1,0,Have been using the product for a week now
2,0,Will continue to use this product when I have ...
3,0,Have always had good luck with this product .
4,0,Will continue to use This product as This prod...
...,...,...
599,10,Spray has no strong odor
600,10,Spray is nice to keep out on the porch on a su...
601,10,Spray does not leave any oily stinky stains
602,10,Love that the scent of this spray is not chemi...


In [16]:
clean_Text = []
for review in df['phrase']:
    clean_Text.append( " ".join(cleaning(review)))

In [22]:
df['clean_Text']=clean_Text

In [23]:
df

,cluster_id,phrase,clean_Text
0,0,Would use the product again if needed Joe .,would use product need joe
1,0,Have been using the product for a week now,use product week
2,0,Will continue to use this product when I have ...,continu use product issu
3,0,Have always had good luck with this product .,alway good luck product
4,0,Will continue to use This product as This prod...,continu use product product get job done
...,...,...,...
599,10,Spray has no strong odor,spray strong odor
600,10,Spray is nice to keep out on the porch on a su...,spray nice keep porch summer day
601,10,Spray does not leave any oily stinky stains,spray leav oili stinki stain
602,10,Love that the scent of this spray is not chemi...,love scent spray chemic smell


 ## One Hot Representation

In [48]:
### Vocabulary size
voc_size=10000
onehot_repr=[one_hot(words,voc_size)for words in sent] 
print(onehot_repr)
sent_length=13
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[472, 3433, 125, 3152, 2186, 6328, 3443, 8419], [687, 1153, 8283, 125, 3152, 7211, 3932, 4374, 918], [5096, 6675, 4029, 3433, 4836, 3152, 9454, 2041, 687, 3932, 6003], [687, 3378, 1806, 6097, 4825, 661, 4836, 3152], [5096, 6675, 4029, 3433, 4836, 3152, 2752, 4836, 3152, 3780, 125, 5055, 1171], [687, 9741, 4029, 3433, 2475, 5122], [5096, 3433, 4836, 3152, 2752, 8112, 2752, 6366], [472, 4016, 3433, 4836, 3152, 2186], [7211, 5794, 3033, 3433, 125, 3152, 6130, 3932, 6097, 6644], [3756, 4029, 3433, 3932, 3152], [3962, 3433, 4836, 5122], [2839, 8283, 4836, 3152], [687, 1153, 8283, 4836, 3152, 7211, 4935], [687, 1153, 8283, 4836, 3152, 7211, 3932, 8910], [687, 4591, 4836, 3152, 7211, 3932, 5620, 2319], [1153, 8283, 3932, 8051, 5988, 918], [687, 4591, 7211, 918, 903, 5988], [687, 1153, 8283, 7211, 6497, 7216, 5988], [687, 1153, 8283, 7402, 7211, 5988], [6722, 4029, 6675, 8283], [2041, 687, 8603, 3433], [687, 4591, 7211, 5683, 5988], [687, 4591, 5683, 2319], [687, 4591, 5733, 2688, 7494], [115

## Word Embedding Reresentation

In [46]:
dim = 10
model=Sequential()
model.add(Embedding(voc_size,10,input_length=sent_length))
model.compile('adam','mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 13, 10)            100000    
                                                                 
Total params: 100,000
Trainable params: 100,000
Non-trainable params: 0
_________________________________________________________________


In [47]:
print(model.predict(embedded_docs))

[[[-8.68175179e-03 -2.56291628e-02  3.91148441e-02 ...  3.45027931e-02
    3.66828553e-02  2.16686167e-02]
  [-8.68175179e-03 -2.56291628e-02  3.91148441e-02 ...  3.45027931e-02
    3.66828553e-02  2.16686167e-02]
  [-8.68175179e-03 -2.56291628e-02  3.91148441e-02 ...  3.45027931e-02
    3.66828553e-02  2.16686167e-02]
  ...
  [-2.49892008e-02 -2.33349688e-02  3.23642977e-02 ...  3.98224853e-02
    2.71653868e-02  3.38083990e-02]
  [ 1.09354630e-02 -1.65530443e-02  2.37953924e-02 ... -1.89642198e-02
    2.04466917e-02 -1.84949748e-02]
  [ 1.05901957e-02 -2.72707231e-02 -1.35453716e-02 ... -3.15380692e-02
   -1.46338344e-02  2.82392241e-02]]

 [[-8.68175179e-03 -2.56291628e-02  3.91148441e-02 ...  3.45027931e-02
    3.66828553e-02  2.16686167e-02]
  [-8.68175179e-03 -2.56291628e-02  3.91148441e-02 ...  3.45027931e-02
    3.66828553e-02  2.16686167e-02]
  [-8.68175179e-03 -2.56291628e-02  3.91148441e-02 ...  3.45027931e-02
    3.66828553e-02  2.16686167e-02]
  ...
  [-4.11644578e-05 -1.0

## Using PCA & t-SNE for reducing dimensions

In [30]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca_result = pca.fit_transform(embedded_docs)

print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

Explained variation per principal component: [0.26944532 0.12165308 0.1064339 ]


In [32]:
from sklearn.manifold import TSNE

In [33]:
tsne = TSNE(n_components=3, verbose=0, perplexity=40, n_iter=300)
tsne_result = tsne.fit_transform(embedded_docs)

## Using DBSCAN

In [34]:
from sklearn.cluster import DBSCAN

In [35]:
model_pca = DBSCAN(eps = 3, min_samples = 5)
model_pca.fit(pca_result)

DBSCAN(eps=3)

In [36]:
model_tsne = DBSCAN(eps = 3, min_samples = 6)
model_tsne.fit(tsne_result)

DBSCAN(eps=3, min_samples=6)

In [37]:
df['PCA-DBSCAN Label'] = model_pca.labels_
df.head(100)

,cluster_id,phrase,clean_Text,PCA-DBSCAN Label
0,0,Would use the product again if needed Joe .,would use product need joe,-1
1,0,Have been using the product for a week now,use product week,-1
2,0,Will continue to use this product when I have ...,continu use product issu,-1
3,0,Have always had good luck with this product .,alway good luck product,-1
4,0,Will continue to use This product as This prod...,continu use product product get job done,-1
...,...,...,...,...
95,1,An amazing value,amaz valu,-1
96,1,Great value as well .,great valu well,-1
97,1,Is a good value .,good valu,-1
98,1,Very good value,good valu,-1


In [38]:
df['tSNE-DBSCAN Label'] = model_tsne.labels_
df.head(100)

,cluster_id,phrase,clean_Text,PCA-DBSCAN Label,tSNE-DBSCAN Label
0,0,Would use the product again if needed Joe .,would use product need joe,-1,-1
1,0,Have been using the product for a week now,use product week,-1,-1
2,0,Will continue to use this product when I have ...,continu use product issu,-1,0
3,0,Have always had good luck with this product .,alway good luck product,-1,-1
4,0,Will continue to use This product as This prod...,continu use product product get job done,-1,0
...,...,...,...,...,...
95,1,An amazing value,amaz valu,-1,0
96,1,Great value as well .,great valu well,-1,0
97,1,Is a good value .,good valu,-1,0
98,1,Very good value,good valu,-1,0


In [39]:
df.groupby(['PCA-DBSCAN Label'])['phrase'].count()

PCA-DBSCAN Label
-1    604
Name: phrase, dtype: int64

In [40]:
df.groupby(['tSNE-DBSCAN Label'])['phrase'].count()

tSNE-DBSCAN Label
-1     66
 0    538
Name: phrase, dtype: int64